# Convnets - Convolutional Neural Networks

**Convolutional Neural Networks (CNNs)** are a class of deep learning models specifically designed for processing structured grid data, such as images. They are particularly effective for tasks like image classification, object detection, and segmentation.

We start with an example to illustrate the basic components of a CNN.  The following code demonstrates a simple CNN that classifies images from the MNIST dataset of handwritten digits. The model consists of several convolutional layers followed by fully connected layers.


In [1]:
from tensorflow import keras 
from tensorflow.keras import layers 

inputs = keras.Input(shape=(28, 28, 1))

# Convolutional layers
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)

# Flattening before Dense layers
x = layers.Flatten()(x)

# Dense (fully-connected) output layer
outputs = layers.Dense(10, activation="softmax")(x)

# Define the model
model = keras.Model(inputs=inputs, outputs=outputs)

# View the model structure clearly
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

In [2]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255 
test_images = test_images.reshape((10000, 28, 28, 1)) 
test_images = test_images.astype("float32") / 255

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)



Epoch 1/5


  1/938 ━━━━━━━━━━━━━━━━━━━━ 4:00 256ms/step - accuracy: 0.0938 - loss: 2.3121

  9/938 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.2119 - loss: 2.2086    

 17/938 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.2995 - loss: 2.0344

 25/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.3649 - loss: 1.8736

...

 52/938 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9975 - loss: 0.0102

 59/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9973 - loss: 0.0105

 65/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9973 - loss: 0.0107

 72/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9972 - loss: 0.0109

KeyboardInterrupt: 

In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print("Test loss ", test_loss)
print("Test accuracy ", test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9911 - loss: 0.0288
Test loss  0.021373014897108078
Test accuracy  0.9936000108718872
